https://blog.csdn.net/qiqi123i/article/details/90022799   
http://neo4j.com.cn/topic/5fd1784cd4a2c822637223a2

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_excel('./理财产品属性.xlsx', index_col=0)

In [3]:
data.columns

Index(['产品名称', '产品代码', '产品登记编码', '本金及理财收益', '巨额赎回', '认购/申购上限', '理财计划到期日',
       '认购、申购及赎回方式', '理财币种', '产品类型', '募集方式', '运作方式', '业绩比较基准', '理财计划份额',
       '理财计划份额面值', '发行规模', '认购起点', '认购期', '认购登记日', '理财计划成立日', '估值日',
       '理财计划申购和赎回', '费用', '税款', '投资策略'],
      dtype='object')

In [4]:
data_columns = list(data.columns)
labels = {
    '名称':data_columns[:3],
    '属性':data_columns[8:12] + [data_columns[23]],
    '发行':data_columns[13:16],
    '收益':[data_columns[3], data_columns[12] ,data_columns[20]],
    '认购申购与赎回':data_columns[4:6] + [data_columns[7]] +data_columns[16:19] + data_columns[21:23] ,
    '开始和结束时间':[data_columns[6], data_columns[19]],
    '投资':data_columns[24],
}

In [5]:
labels

{'名称': ['产品名称', '产品代码', '产品登记编码'],
 '属性': ['理财币种', '产品类型', '募集方式', '运作方式', '税款'],
 '发行': ['理财计划份额', '理财计划份额面值', '发行规模'],
 '收益': ['本金及理财收益', '业绩比较基准', '估值日'],
 '认购申购与赎回': ['巨额赎回',
  '认购/申购上限',
  '认购、申购及赎回方式',
  '认购起点',
  '认购期',
  '认购登记日',
  '理财计划申购和赎回',
  '费用'],
 '开始和结束时间': ['理财计划到期日', '理财计划成立日'],
 '投资': '投资策略'}

In [6]:
def change_labels(labels):
    clabels = {}
    for l in labels:
        if type(labels[l]) == list:
            for n in labels[l]:
                clabels[n] = l
        else:
            clabels[labels[l]] = l
    return clabels

clabels = change_labels(labels)
clabels

{'产品名称': '名称',
 '产品代码': '名称',
 '产品登记编码': '名称',
 '理财币种': '属性',
 '产品类型': '属性',
 '募集方式': '属性',
 '运作方式': '属性',
 '税款': '属性',
 '理财计划份额': '发行',
 '理财计划份额面值': '发行',
 '发行规模': '发行',
 '本金及理财收益': '收益',
 '业绩比较基准': '收益',
 '估值日': '收益',
 '巨额赎回': '认购申购与赎回',
 '认购/申购上限': '认购申购与赎回',
 '认购、申购及赎回方式': '认购申购与赎回',
 '认购起点': '认购申购与赎回',
 '认购期': '认购申购与赎回',
 '认购登记日': '认购申购与赎回',
 '理财计划申购和赎回': '认购申购与赎回',
 '费用': '认购申购与赎回',
 '理财计划到期日': '开始和结束时间',
 '理财计划成立日': '开始和结束时间',
 '投资策略': '投资'}

In [7]:
data.head(5)

,产品名称,产品代码,产品登记编码,本金及理财收益,巨额赎回,认购/申购上限,理财计划到期日,认购、申购及赎回方式,理财币种,产品类型,...,发行规模,认购起点,认购期,认购登记日,理财计划成立日,估值日,理财计划申购和赎回,费用,税款,投资策略
0,招商银行年年享5号理财计划,107205,C1030819000925,本理财计划不保证本金且不保证理财收益，收益随投资收益浮动。,投资者可于赎回日内向招商银行提出赎回申请，若理财计划每日净赎回额超过本理财计划上一工作日份额...,5000万元,2049年7月12日,个人投资者可通过招商银行各营业网点、招商银行财富账户、个人银行专业版、大众版或者招商银行认可...,人民币,固定收益类,...,1亿份-20亿份,1万份,2019年7月4日10:00-2019年7月11日17：00,2019年7月12日,2019年7月12日,本理财计划存续期内，每周三及开放日（或理财计划实际到期日）为估值日，如周三为非工作日，则估值...,本理财计划每12个月为一个投资周期，投资者可在每个投资周期结束日前7个工作日（含）至该投资周...,1.认/申购费：本理财计划不收取认/申购费。2.固定投资管理费：招商银行收取理财计划固定投资...,本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求...,"<table border=""1"" cellspacing=""0"" class=""dataf..."
1,招商银行年年享6号理财计划,107206,C1030819001083,本理财计划不保证本金且不保证理财收益，收益随投资收益浮动。,投资者可于赎回日内向招商银行提出赎回申请，若理财计划每日净赎回额超过本理财计划上一工作日份额...,5000万元,2049年8月13日,个人投资者可通过招商银行各营业网点、招商银行财富账户、个人银行专业版、大众版或者招商银行认可...,人民币,固定收益类,...,1亿份-30亿份,1万份,2019年8月5日10:00-2019年8月12日17：00,2019年8月13日,2019年8月13日,本理财计划存续期内，每周三及开放日（或理财计划实际到期日）为估值日，如周三为非工作日，则估值...,本理财计划每12个月为一个投资周期，投资者可在每个投资周期结束日前7个工作日（含）至该投资周...,1.认/申购费：本理财计划不收取认/申购费。2.固定投资管理费：招商银行收取理财计划固定投资...,本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求...,"<table border=""1"" cellspacing=""0"" class=""dataf..."
2,招商银行美元金葵一年定开1号理财计划,2101,C1030819001500,本理财计划不保障本金且不保证理财收益，收益随投资收益浮动。,投资者可于赎回日内向招商银行提出赎回申请，在理财计划存续期内的每个赎回日，若理财计划净赎回份...,1000万美元,2029年11月7日,个人投资者可通过招商银行各营业网点、招商银行财富账户、个人银行专业版、大众版或者招商银行认可...,美元,固定收益类,...,2000万份-3亿份,2000份,2019年10月30日10:00-2019年11月6日17:00,2019年11月7日,2019年11月7日,本理财计划存续期内，每周一及开放日（或理财计划实际到期日）为估值日，如周一为非交易日，则估值...,本理财计划每12个月为一个投资周期，投资者可在申购、赎回期提交本理财计划的申购、赎回申请。一...,1.认/申购费：本理财计划不收取认/申购费。2.固定投资管理费：招商银行收取理财计划固定投资...,本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求...,"<table border=""1"" cellspacing=""0"" class=""dataf..."
3,招商银行美元金葵一年定开2号理财计划,2103,C1030819001689,本理财计划不保障本金且不保证理财收益，收益随投资收益浮动。,投资者可于赎回日内向招商银行提出赎回申请，在理财计划存续期内的每个赎回日，若理财计划净赎回份...,1000万美元,2029年12月11日,个人投资者可通过招商银行各营业网点、招商银行财富账户、个人银行专业版、大众版或者招商银行认可...,美元,固定收益类,...,2000万份-3亿份,2000份,2019年12月4日10:00-2019年12月10日17:00,2019年12月11日,2019年12月11日,本理财计划存续期内，每周一及开放日（或理财计划实际到期日）为估值日，如周一为非交易日，则估值...,本理财计划每12个月为一个投资周期，投资者可在申购、赎回期提交本理财计划的申购、赎回申请。一...,1.认/申购费：本理财计划不收取认/申购费。2.固定投资管理费：招商银行收取理财计划固定投资...,本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求...,"<table border=""1"" cellspacing=""0"" class=""dataf..."
4,招商银行美元金葵一年定开3号理财计划,2105,C1030820000012,本理财计划不保障本金且不保证理财收益，收益随投资收益浮动。,投资者可于赎回日内向招商银行提出赎回申请，在理财计划存续期内的每个赎回日，若理财计划净赎回份...,1000万美元,2030年1月13日,个人投资者可通过招商银行各营业网点、招商银行财富账户、个人银行专业版、大众版或者招商银行认可...,美元,固定收益类,...,2000万份-3亿份,2000份,2020年1月2日10:00-2020年1月12日17:00,2020年1月13日,2020年1月13日,本理财计划存续期内，每周一及开放日（或理财计划实际到期日）为估值日，如周一为非交易日，则估值...,本理财计划每12个月为一个投资周期，投资者可在申购、赎回期提交本理财计划的申购、赎回申请。一...,1.认/申购费：本理财计划不收取认/申购费。2.固定投资管理费：招商银行收取理财计划固定投资...,本理财计划运作过程中涉及的各纳税主体，其纳税义务按国家税收法律、法规执行。除法律法规特别要求...,"<table border=""1"" cellspacing=""0"" class=""dataf..."


In [8]:
data.isna().sum()

产品名称           0
产品代码           0
产品登记编码         0
本金及理财收益        0
巨额赎回          30
认购/申购上限        0
理财计划到期日        0
认购、申购及赎回方式     0
理财币种           0
产品类型           0
募集方式           0
运作方式           0
业绩比较基准         0
理财计划份额         0
理财计划份额面值       0
发行规模           0
认购起点           0
认购期            0
认购登记日          0
理财计划成立日        0
估值日            0
理财计划申购和赎回      0
费用             0
税款             0
投资策略           0
dtype: int64

In [9]:
# 将null值填充为''
data.replace(np.nan,'',inplace=True)

In [10]:
data.isna().sum()

产品名称          0
产品代码          0
产品登记编码        0
本金及理财收益       0
巨额赎回          0
认购/申购上限       0
理财计划到期日       0
认购、申购及赎回方式    0
理财币种          0
产品类型          0
募集方式          0
运作方式          0
业绩比较基准        0
理财计划份额        0
理财计划份额面值      0
发行规模          0
认购起点          0
认购期           0
认购登记日         0
理财计划成立日       0
估值日           0
理财计划申购和赎回     0
费用            0
税款            0
投资策略          0
dtype: int64

In [11]:
def create_search_df(df):
    search = []
    for l in df.columns:
        if l not in ['产品代码', '产品登记编码']:
            d = df[l].drop_duplicates().values
            search.append(list(zip(d,[l]*d.shape[0])))
    search = sum(search,[])
    search = pd.DataFrame(search, columns=['value','name'])
    search = search.reset_index(drop=False,inplace=False).rename(columns={'index':'id:ID'})
    return search

In [12]:
search_df = create_search_df(data)
search_df

,id:ID,value,name
0,0,招商银行年年享5号理财计划,产品名称
1,1,招商银行年年享6号理财计划,产品名称
2,2,招商银行美元金葵一年定开1号理财计划,产品名称
3,3,招商银行美元金葵一年定开2号理财计划,产品名称
4,4,招商银行美元金葵一年定开3号理财计划,产品名称
...,...,...,...
319,319,"<table border=""1"" cellspacing=""0"" class=""dataf...",投资策略
320,320,"<table border=""1"" cellspacing=""0"" class=""dataf...",投资策略
321,321,"<table border=""1"" cellspacing=""0"" class=""dataf...",投资策略
322,322,"<table border=""1"" cellspacing=""0"" class=""dataf...",投资策略


In [13]:
labels

{'名称': ['产品名称', '产品代码', '产品登记编码'],
 '属性': ['理财币种', '产品类型', '募集方式', '运作方式', '税款'],
 '发行': ['理财计划份额', '理财计划份额面值', '发行规模'],
 '收益': ['本金及理财收益', '业绩比较基准', '估值日'],
 '认购申购与赎回': ['巨额赎回',
  '认购/申购上限',
  '认购、申购及赎回方式',
  '认购起点',
  '认购期',
  '认购登记日',
  '理财计划申购和赎回',
  '费用'],
 '开始和结束时间': ['理财计划到期日', '理财计划成立日'],
 '投资': '投资策略'}

In [14]:
def handle_out(x):
    if x == '':
        return x
    else:
        if '万份' in x:
            return str(int(x[:-2])*10000)
        else:
            return str(int(x[:-2])*100000000)

def create_node_csv(search, labels, data_df):
    path = './实体/'
    for label in labels:
        print('正在写入 {}'.format(label))
        if label == '名称':
            df = search[search['name']==labels[label][0]]
            df = df.rename(columns={'id:ID':'id:ID({})'.format(label)})
            df = pd.merge(df, data_df[['产品名称', '产品代码']], left_on='value', right_on='产品名称')
            df = pd.merge(df, data_df[['产品名称', '产品登记编码']], left_on='value', right_on='产品名称')
            df = df[['id:ID(名称)', 'value', '产品代码', '产品登记编码']]
            df.columns = ['id:ID(名称)', 'name', 'code', 'outer_code']
        elif label == '发行':
            df = search[search['name'].isin(labels[label])]
            df = df.drop(df[df['name']=='发行规模'].index)
            df1 = search[search['name']=='发行规模']
            df1['up'] = df1['value'].map(lambda x: x.split('-')[1])
            df1['down'] = df1['value'].map(lambda x: x.split('-')[0])
            df1['up'] = df1['up'].map(lambda x: handle_out(x))
            df1['down'] = df1['down'].map(lambda x: handle_out(x))
            df = df.append(df1)
        elif label == '收益':
            # 业绩比较基准
            df = search[search['name'].isin(labels[label])]
            df = df.drop(df[df['name']=='业绩比较基准'].index)
            df1 = search[search['name']=='业绩比较基准']
            df1['up'] = df1['value'].map(lambda x: x.split('-')[1][:-1])
            df1['down'] = df1['value'].map(lambda x: x.split('-')[0][:-1])
            df = df.append(df1)
        elif label == '认购申购与赎回':
            df = search[search['name'].isin(labels[label])]
            # 认购/申购上限
            df = df.drop(df[df['name']=='认购/申购上限'].index)
            df1 = search[search['name']=='认购/申购上限']
            df1['unit'] = df1['value'].map(lambda x: '美元' if '美元' in x else '元')
            df1['up'] = df1['value'].map(lambda x: str(int(x[:-3])*10000) if '美元' in x else str(int(x[:-2])*10000))
            df = df.append(df1)
            # 认购起点
            df = df.drop(df[df['name']=='认购起点'].index)
            df1 = search[search['name']=='认购起点']
            df1['down'] = df1['value'].map(lambda x: str(int(x[:-2])*10000) if '万份' in x else str(int(x[:-1])))
            df1['unit'] = '份'
            df = df.append(df1)
            # 认购期
            df = df.drop(df[df['name']=='认购期'].index)
            df1 = search[search['name']=='认购期']
            df1['up'] = df1['value'].map(lambda x: x.split('-')[1])
            df1['down'] = df1['value'].map(lambda x: x.split('-')[0])
            df1['up'] = df1['up'].map(lambda x: x[:x.index('年')] + x[x.index('年')+1: x.index('月')].zfill(2) + \
                                           x[x.index('月')+1: x.index('日')].zfill(2))
            df1['down'] = df1['down'].map(lambda x: x[:x.index('年')] + x[x.index('年')+1: x.index('月')].zfill(2) + \
                                           x[x.index('月')+1: x.index('日')].zfill(2))
            df = df.append(df1)
            # 认购登记日
            df = df.drop(df[df['name']=='认购登记日'].index)
            df1 = search[search['name']=='认购登记日']
            df1['value'] = df1['value'].map(lambda x: x[:x.index('年')] + x[x.index('年')+1: x.index('月')].zfill(2) + \
                                           x[x.index('月')+1: x.index('日')].zfill(2))
            df = df.append(df1)
            # 理财计划申购和赎回
            df = df.drop(df[df['name']=='理财计划申购和赎回'].index)
            df1 = search[search['name']=='理财计划申购和赎回']
            pattern = re.compile(r'[\u4e00-\u9fa5\0-9]+')
            df1['period'] = df1['value'].map(lambda x: pattern.findall(x)[0])
            pattern1 = re.compile(r'[0-9][\u4e00-\u9fa5\0-9]+月')
            pattern2 = re.compile(r'每[\u4e00-\u9fa5\0-9]*年')
            df1['period'] = df1['period'].map(lambda x: pattern1.findall(x)[0] if '月' in x else pattern2.findall(x)[0])
            df1['period'] = df1['period'].map(lambda x: '12个月' if '每年' == x else x)
            df1['unit'] = df1['period'].map(lambda x: x[-1])
            df1['period'] = df1['period'].map(lambda x: x[:-2])
            df = df.append(df1)
        elif label == '开始和结束时间':
            df = search[search['name'].isin(labels[label])]
            # 理财计划到期日
            df = df.drop(df[df['name']=='理财计划到期日'].index)
            df1 = search[search['name']=='理财计划到期日']
            df1['value'] = df1['value'].map(lambda x: x[:x.index('年')] + x[x.index('年')+1: x.index('月')].zfill(2) + \
                                           x[x.index('月')+1: x.index('日')].zfill(2))
            df = df.append(df1)
            # 理财计划成立日
            df = df.drop(df[df['name']=='理财计划成立日'].index)
            df1 = search[search['name']=='理财计划成立日']
            df1['value'] = df1['value'].map(lambda x: x[:x.index('年')] + x[x.index('年')+1: x.index('月')].zfill(2) + \
                                           x[x.index('月')+1: x.index('日')].zfill(2))
            df = df.append(df1)
        else:
            if type(labels[label]) == list:
                df = search[search['name'].isin(labels[label])]
            else:
                df = search[search['name']==labels[label]]
        
        df = df.rename(columns={'id:ID':'id:ID({})'.format(label)})
        file_name = path + label + '.csv'
        df.to_csv(file_name,index=0)

In [15]:
create_node_csv(search_df, labels, data)

正在写入 名称
正在写入 属性
正在写入 发行
正在写入 收益
正在写入 认购申购与赎回
正在写入 开始和结束时间
正在写入 投资


In [16]:
def create_relationship(df, search, labels):
    idx = list(df.columns)[0]
    path = './关系/'
    for l in labels:
        if l != '名称':
            print('正在写入 {}'.format(l))
            file_name = path + l + '_rel' + '.csv'
            l_ = labels[l]
            if type(l_) == list:
                d = pd.merge(df[idx],df[l_], right_index=True, left_index=True)
                result = pd.DataFrame()
                for ll in l_:
                    ll_r = pd.DataFrame()
                    ll_r[':START_ID(名称)'] = df[idx].map(lambda x: search[search['value']==x]['id:ID'].values[0])
                    ll_r['links'] = ll + '为'
                    ll_r[':END_ID({})'.format(l)] = d[ll].map\
                                (lambda x: search[(search['name']==ll)&(search['value']==x)]['id:ID'].values[0])
                    result = result.append(ll_r)
                result.to_csv(file_name,index=0)
            else:
                d = df[[idx,l_]]
                d[':END_ID'] = d[l_].map(lambda x: search[(search['name']==l_)&(search['value']==x)]['id:ID'].values[0])
                d['links'] = l_ + '为'
                d = d[[idx,'links',':END_ID']]
                d[idx] = d[idx].map(lambda x: search[search['value']==x]['id:ID'].values[0])
                d.columns = [':START_ID(名称)','links',':END_ID({})'.format(l)]
                d.to_csv(file_name,index=0)

In [17]:
create_relationship(data, search_df, labels)

正在写入 属性
正在写入 发行
正在写入 收益
正在写入 认购申购与赎回
正在写入 开始和结束时间
正在写入 投资


In [18]:
start = 'neo4j-admin import --database test --id-type=STRING'
node = '--nodes='
relation = '--relationships='
lines = []
for l in labels:
    line = node + l + '=' + '/home/ubuntu/neo4j/import/实体/' + l + '.csv'
    lines.append(line)
for l in labels:
    if l != '名称':
        line = relation + l + '=' + '/home/ubuntu/neo4j/import/关系/' + l + '_rel.csv'
        lines.append(line)

In [19]:
start+' '+' '.join(lines)

'neo4j-admin import --database test --id-type=STRING --nodes=名称=/home/ubuntu/neo4j/import/实体/名称.csv --nodes=属性=/home/ubuntu/neo4j/import/实体/属性.csv --nodes=发行=/home/ubuntu/neo4j/import/实体/发行.csv --nodes=收益=/home/ubuntu/neo4j/import/实体/收益.csv --nodes=认购申购与赎回=/home/ubuntu/neo4j/import/实体/认购申购与赎回.csv --nodes=开始和结束时间=/home/ubuntu/neo4j/import/实体/开始和结束时间.csv --nodes=投资=/home/ubuntu/neo4j/import/实体/投资.csv --relationships=属性=/home/ubuntu/neo4j/import/关系/属性_rel.csv --relationships=发行=/home/ubuntu/neo4j/import/关系/发行_rel.csv --relationships=收益=/home/ubuntu/neo4j/import/关系/收益_rel.csv --relationships=认购申购与赎回=/home/ubuntu/neo4j/import/关系/认购申购与赎回_rel.csv --relationships=开始和结束时间=/home/ubuntu/neo4j/import/关系/开始和结束时间_rel.csv --relationships=投资=/home/ubuntu/neo4j/import/关系/投资_rel.csv'